In [109]:
import pandas as pd

In [110]:
df=pd.read_csv("spotify_millsongdata.csv")

In [111]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [112]:
df=df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [113]:
df

,artist,song,text
0,Bob Dylan,Everything Is Broken,"Broken lines, broken strings, \r\nBroken thre..."
1,Leann Rimes,Right Kind Of Wrong,Know all about \r\nAbout your reputation \r\...
2,Beach Boys,Hushabye,Oo ooo ooo ooo ooo ooo ooo \r\nAh ah ah ah \...
3,Insane Clown Posse,Rollin' Over,"Rolin' over, Hell muthafuckin' yeah bitch. Hat..."
4,Red Hot Chili Peppers,Blues For Meister,It's always half and half \r\nI'm happy and I...
...,...,...,...
4995,Nickelback,Falls Back On,"No, I don't guess \r\nI don't sacrifice \r\n..."
4996,Uncle Tupelo,So Called Friend,"Never again, your so called friend \r\nWill l..."
4997,Peter Cetera,I Can Feel It,You know there's something I gotta say \r\nI ...
4998,Bob Seger,Like A Rock,"Stood there boldly \r\nSweatin', in the sun ..."


In [114]:
df['text'][0]

"Broken lines, broken strings,  \r\nBroken threads, broken springs,  \r\nBroken idols, broken heads,  \r\nPeople sleeping in broken beds.  \r\nAin't no use jiving  \r\nAin't no use joking  \r\nEverything is broken.  \r\n  \r\nBroken bottles, broken plates,  \r\nBroken switches, broken gates,  \r\nBroken dishes, broken parts,  \r\nStreets are filled with broken hearts.  \r\nBroken words never meant to be spoken,  \r\nEverything is broken.  \r\n  \r\nBridge: Seem like every time you stop and turn around  \r\nSomething else just hit the ground  \r\n  \r\nBroken cutters, broken saws,  \r\nBroken buckles, broken laws,  \r\nBroken bodies, broken bones,  \r\nBroken voices on broken phones.  \r\nTake a deep breath, feel like you're chokin',  \r\nEverything is broken.  \r\n  \r\nBridge: Every time you leave and go off someplace  \r\nThings fall to pieces in my face  \r\n  \r\nBroken hands on broken ploughs,  \r\nBroken treaties, broken vows,  \r\nBroken pipes, broken tools,  \r\nPeople bending 

In [115]:
df.shape

(5000, 3)

In [116]:
df['text']=df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ',regex=True)

In [117]:
df

,artist,song,text
0,Bob Dylan,Everything Is Broken,"broken lines, broken strings, \r broken threa..."
1,Leann Rimes,Right Kind Of Wrong,know all about \r about your reputation \r a...
2,Beach Boys,Hushabye,oo ooo ooo ooo ooo ooo ooo \r ah ah ah ah \r...
3,Insane Clown Posse,Rollin' Over,"rolin' over, hell muthafuckin' yeah bitch. hat..."
4,Red Hot Chili Peppers,Blues For Meister,it's always half and half \r i'm happy and i'...
...,...,...,...
4995,Nickelback,Falls Back On,"no, i don't guess \r i don't sacrifice \r an..."
4996,Uncle Tupelo,So Called Friend,"never again, your so called friend \r will le..."
4997,Peter Cetera,I Can Feel It,you know there's something i gotta say \r i c...
4998,Bob Seger,Like A Rock,"stood there boldly \r sweatin', in the sun \..."


In [118]:
import nltk
from nltk.stem.porter import PorterStemmer

In [119]:
stemmer=PorterStemmer()

In [120]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(w) for w in token]
    return " ".join(a)

In [121]:
token("you are beautiful , beauty")

'you are beauti , beauti'

In [122]:
df['text'].apply(lambda x:token(x))

0       broken line , broken string , broken thread , ...
1       know all about about your reput and how it 's ...
2       oo ooo ooo ooo ooo ooo ooo ah ah ah ah oo ooo ...
3       rolin ' over , hell muthafuckin ' yeah bitch ....
4       it 's alway half and half i 'm happi and i 'm ...
                              ...                        
4995    no , i do n't guess i do n't sacrific and i kn...
4996    never again , your so call friend will leav yo...
4997    you know there 's someth i got ta say i can fe...
4998    stood there boldli sweatin ' , in the sun felt...
4999    ding dong ! the witch is dead . which old witc...
Name: text, Length: 5000, dtype: object

In [123]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [124]:
tfid=TfidfVectorizer(analyzer='word',stop_words='english')

In [125]:
matrix=tfid.fit_transform(df['text'])

In [126]:
similar = cosine_similarity(matrix)

In [127]:
similar[1]

array([0.00792112, 1.        , 0.00160354, ..., 0.17560434, 0.02786788,
       0.01087578])

In [139]:
def recommender(song_name):
        idx = df[df['song'] == song_name].index[0]
        dist = sorted(list(enumerate(similar[idx])), reverse=True, key=lambda x: x[1])
        song = []
        for s_id in dist[1:21]:
            song.append(df.iloc[s_id[0]].song)
        return song


In [140]:
recommender("Hushabye")

['Not Following You',
 'Island Fever',
 'Train Leaves Here This Morning',
 'Echo',
 'Born To Die',
 "We'll Be Alright",
 'Fallin Down',
 'Make Me A Believer',
 'Smile At You',
 "Witches' Brew",
 'Look At Me Now',
 'If I Needed Someone',
 'Senorita',
 'Leave This Man Alone',
 "You're The One That I Want",
 'Weeping',
 'Let Me Down Easy',
 'Staring At The Sun',
 'Stayin Alive (Rare)',
 'Forever You Will Be Mine']

In [141]:
import pickle

In [142]:
pickle.dump(similar,open("similarity","wb"))

In [143]:
pickle.dump(df ,open("df","wb"))